In [3]:
import pandas as pd
import re
import string  # Import the string module for punctuations

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Load the training data
train = pd.read_csv('traintweet.csv')

# Convert tweets to lowercase
train['tweet'] = train['tweet'].str.lower()

# Define function to remove punctuations
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

# Apply punctuations cleaning function to tweets
train['tweet'] = train['tweet'].apply(lambda x: cleaning_punctuations(x))

# Define function to clean email addresses
def cleaning_email(data):
    return re.sub('@[^\s]+', ' ', data)

# Apply email cleaning function to tweets
train['tweet'] = train['tweet'].apply(lambda x: cleaning_email(x))

# Define function to clean URLs
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', data)

# Apply URL cleaning function to tweets
train['tweet'] = train['tweet'].apply(lambda x: cleaning_URLs(x))

# Define function to clean numbers
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)

# Apply number cleaning function to tweets
train['tweet'] = train['tweet'].apply(lambda x: cleaning_numbers(x))

# Split the data into features (X) and labels (y)
X = train['tweet']
y = train['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Create a pipeline with TfidfVectorizer and LinearSVC
text_clf_lsvc = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

# Train the classifier
text_clf_lsvc.fit(X_train, y_train)

# Evaluate the classifier
accuracy = text_clf_lsvc.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.9611300720515737


In [4]:
# Assuming you have already loaded and preprocessed the test data
test = pd.read_csv('testtweet.csv')
test['tweet'] = test['tweet'].str.lower()
test['tweet'] = test['tweet'].apply(lambda x: cleaning_punctuations(x))
test['tweet'] = test['tweet'].apply(lambda x: cleaning_email(x))
test['tweet'] = test['tweet'].apply(lambda x: cleaning_URLs(x))
test['tweet'] = test['tweet'].apply(lambda x: cleaning_numbers(x))

# Get the features (X_test) for the test data
X_test = test['tweet']

# Predict labels for the test data
predicted_labels = text_clf_lsvc.predict(X_test)

# Assuming 'predicted_labels' is a numpy array containing the predicted labels
# You can create a DataFrame with the predicted labels
predicted_df = pd.DataFrame({'label': predicted_labels})

# Now you can do whatever you want with the predicted labels DataFrame
# For example, you can save it to a CSV file
predicted_df.to_csv('predicted_labels.csv', index=False)
